In [1]:
with open('data/aoc-2021-16.txt') as f:
    dat = f.readlines()[0].strip()

In [3]:
format(10,'04b')

'1010'

In [2]:
dat

'420D4900B8F31EFE7BD9DA455401AB80021504A2745E1007A21C1C862801F54AD0765BE833D8B9F4CE8564B9BE6C5CC011E00D5C001098F11A232080391521E4799FC5BB3EE1A8C010A00AE256F4963B33391DEE57DA748F5DCC011D00461A4FDC823C900659387DA00A49F5226A54EC378615002A47B364921C201236803349B856119B34C76BD8FB50B6C266EACE400424883880513B62687F38A13BCBEF127782A600B7002A923D4F959A0C94F740A969D0B4C016D00540010B8B70E226080331961C411950F3004F001579BA884DD45A59B40005D8362011C7198C4D0A4B8F73F3348AE40183CC7C86C017997F9BC6A35C220001BD367D08080287914B984D9A46932699675006A702E4E3BCF9EA5EE32600ACBEADC1CD00466446644A6FBC82F9002B734331D261F08020192459B24937D9664200B427963801A094A41CE529075200D5F4013988529EF82CEFED3699F469C8717E6675466007FE67BE815C9E84E2F300257224B256139A9E73637700B6334C63719E71D689B5F91F7BFF9F6EE33D5D72BE210013BCC01882111E31980391423FC4920042E39C7282E4028480021111E1BC6310066374638B200085C2C8DB05540119D229323700924BE0F3F1B527D89E4DB14AD253BFC30C01391F815002A539BA9C4BADB80152692A012CDCF20F35FDF635A9CCC71F261A080356B0056

In [10]:
def hex2bin(hs):
    bs = ''
    for h in hs:
        bs += format(int(h, 16), '04b')
    return bs

In [16]:
bindat = hex2bin(dat)

In [64]:
def parsePacket(b, i):
    v = int(b[i+0:i+3], 2)
    t = int(b[i+3:i+6], 2)
    i = i + 6
    
    if t == 4:
        ds = ''
        while True:
            more = b[i] == '1'
            ds += b[i+1:i+5]
            i += 5
            if not more:
                break
        d = int(ds, 2)
        return (i, (v, t, d))
    else:
        if b[i] == '0': #15 bits total length
            i += 1
            end = i + 15 + int(b[i:i+15], 2)
            i += 15
            subpkts = []
            while i < end:
                i, pkt = parsePacket(b, i)
                subpkts += [pkt]
            d = subpkts
        else: #11 bit subpacket count
            i += 1
            cnt = int(b[i:i+11], 2)
            i += 11
            subpkts = []
            for f in range(cnt):
                i, pkt = parsePacket(b, i)
                subpkts += [pkt]
            d = subpkts
        return (i, (v, t, d))
    return 0

In [65]:
def verSum(pkt):
    vs = pkt[0]
    if pkt[1] != 4:
        for p in pkt[2]:
            vs += verSum(p)
    return vs

In [66]:
pkts = parsePacket(bindat, 0)[1]
vs = verSum(pkts)
print(f'Part 1: {vs}')

Part 1: 947


In [69]:
from functools import reduce

def evalPacket(pkt):
    if pkt[1] == 4:
        return pkt[2]

    vals = [evalPacket(p) for p in pkt[2]]

    if pkt[1] == 0:
        return sum(vals)
    elif pkt[1] == 1:
        return reduce(lambda a, b: a * b, vals)
    elif pkt[1] == 2:
        return min(vals)
    elif pkt[1] == 3:
        return max(vals)
    elif pkt[1] == 5:
        return 1 if vals[0] > vals[1] else 0
    elif pkt[1] == 6:
        return 1 if vals[0] < vals[1] else 0
    elif pkt[1] == 7:
        return 1 if vals[0] == vals[1] else 0

In [70]:
evalPacket(pkts)

660797830937

In [71]:
pkts

(2,
 0,
 [(1,
   1,
   [(7, 4, 614077380133),
    (2,
     5,
     [(7, 0, [(2, 4, 2), (2, 4, 9), (6, 4, 11)]),
      (6, 0, [(6, 4, 8), (3, 4, 3), (4, 4, 6)])])]),
  (1,
   2,
   [(2, 4, 1347), (5, 4, 1784), (1, 4, 59511), (6, 4, 2466), (5, 4, 2931890)]),
  (7,
   1,
   [(1,
     7,
     [(7, 0, [(1, 4, 15), (0, 4, 13), (0, 4, 12)]),
      (4, 0, [(4, 4, 10), (4, 4, 7), (4, 4, 15)])]),
    (1, 4, 4167908749)]),
  (2, 1, [(0, 5, [(0, 4, 1463979318), (3, 4, 29042617)]), (6, 4, 478)]),
  (7, 1, [(1, 6, [(1, 4, 6), (4, 4, 3956)]), (0, 4, 15)]),
  (1, 1, [(4, 4, 199), (6, 6, [(4, 4, 985), (0, 4, 2722958220)])]),
  (1, 2, [(2, 4, 7484)]),
  (4, 4, 35),
  (4, 1, [(5, 5, [(4, 4, 184877372), (6, 4, 9714442831537)]), (1, 4, 2908)]),
  (7, 1, [(4, 4, 16)]),
  (7, 0, [(0, 4, 3428), (6, 4, 7), (7, 4, 202), (0, 4, 231192622), (7, 4, 2)]),
  (5, 1, [(6, 7, [(4, 4, 473), (7, 4, 1336)]), (1, 4, 146241)]),
  (2, 4, 25413),
  (5,
   1,
   [(6,
     6,
     [(5, 0, [(3, 4, 11), (3, 4, 7), (0, 4, 9)]),
  